In [138]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [139]:
x_train_val = np.load("/kaggle/input/rn2021q1itba-cifar100/x_train.npy")
x_test = np.load("/kaggle/input/rn2021q1itba-cifar100/x_test.npy")
y_train_val = np.load("/kaggle/input/rn2021q1itba-cifar100/y_train.npy")

FileNotFoundError: FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/rn2021q1itba-cifar100/x_train.npy'

In [140]:
from matplotlib import pyplot as plt
#Mostramos alguna imágen
idx = 50
plt.imshow(x_train_val[idx])
plt.title(f'Clase: {y_train_val[idx]}')
plt.show()

NameError: NameError: name 'x_train_val' is not defined

In [141]:
#Dividimos entre train y validación
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(
    x_train_val, y_train_val, test_size=0.15, random_state=42, stratify=y_train_val)

NameError: NameError: name 'x_train_val' is not defined

In [142]:
#Normalizamos
x_train_norm = x_train/255
x_val_norm = x_val/255
x_test_norm = x_test/255

NameError: NameError: name 'x_train' is not defined

In [143]:
#Armamos la red de clasificación
from tensorflow.keras.layers import Dense, Flatten, Activation, BatchNormalization, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam

In [144]:
model = Sequential()
model.add(Conv2D(64,3, input_shape=(32,32,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D())
model.add(Conv2D(128,3))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D())
model.add(Conv2D(256,3))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dense(100, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy", optimizer= Adam(learning_rate=0.001), metrics=["accuracy"])

In [145]:
model.fit(x_train_norm, y_train, batch_size = 256, validation_data = (x_val_norm, y_val), epochs=10)

NameError: NameError: name 'x_train_norm' is not defined

In [146]:
y_test = model.predict(x_test_norm).argmax(axis=-1)

NameError: NameError: name 'x_test_norm' is not defined

In [147]:
y_test

NameError: NameError: name 'y_test' is not defined

In [148]:
#Salvo las predicciones y genero el archivo csv según el formato pedido
df = pd.DataFrame(y_test, columns=["label"])
df.index.name = "Id"
df.to_csv("submission.csv")

NameError: NameError: name 'y_test' is not defined

In [149]:
print('holaaefwasde')
print("holanda")

holaaefwasde
holanda


In [150]:
print('fwgwq')

fwgwq


In [151]:
print('rigne')

rigne


In [152]:
print('fjweijfw')

fjweijfw


In [153]:
print("holandamio")

holandamio
